In [1]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model 
from tensorflow.keras.applications.vgg16 import VGG16 , preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import cv2 
import numpy as np 

In [5]:
input_shape = [32,32,3]
model  = VGG16(input_shape = input_shape , weights = 'imagenet' , include_top = False)

In [6]:
for layers in model.layers : 
  layers.trainable = False 

In [7]:
x = Flatten()(model.output)
output = Dense(10 , activation = 'softmax')(x)

In [8]:
model1 = Model(inputs = model.input , outputs = output)
model1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [9]:
(x_train, y_train), (x_test, y_test) = tensorflow.keras.datasets.cifar10.load_data()

In [10]:
x_train.shape , y_train.shape , x_test.shape , y_test.shape

((50000, 32, 32, 3), (50000, 1), (10000, 32, 32, 3), (10000, 1))

In [11]:
x_train = x_train[0:20000]
y_train = y_train[0:20000]

In [12]:
import pandas as pd 
pd.DataFrame(y_train).value_counts()

2    2042
6    2038
7    2027
8    2012
3    2011
4    2009
0    1989
1    1981
9    1966
5    1925
dtype: int64

In [17]:
model1.compile(optimizer='adam' , loss='categorical_crossentropy' , metrics =['accuracy'])

In [18]:
x_train = np.array([cv2.resize(x , (32,32), interpolation = cv2.INTER_AREA) for x in x_train])
x_test = np.array([cv2.resize(x , (32,32), interpolation = cv2.INTER_AREA) for x in x_test])

In [19]:
train_datagen = ImageDataGenerator(rescale = 1./255)
train_datagen.fit(x_train)
test_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen.fit(x_test)

In [20]:
train_data = train_datagen.flow(x_train,y_train , batch_size = 20)
test_data = test_datagen.flow(x_test , y_test , batch_size = 20)

In [ ]:
history = model1.fit(train_data , validation_data=test_data, epochs = 1000) 

In [ ]:
model1.save('cifar.h5')

In [22]:
new_model = tensorflow.keras.models.load_model('cifar.h5')

In [23]:
img = load_img('truck.jfif' , target_size = (32,32))
img = img_to_array(img)
img = np.expand_dims(img, axis = 0)
img = preprocess_input(img)

In [24]:
prediction = new_model.predict(img)

In [25]:
result = np.argmax(prediction)

In [26]:
labels = ['airplane', 'automobile' ,'bird' ,'cat' ,'deer' ,'dog', 'frog' ,'horse', 'ship' ,'truck']

In [27]:
labels[result]

'truck'